In [58]:
# base_path = './'

In [59]:
import pandas as pd
from time import time
import tensorflow as tf
import os

In [60]:
def gerar_rede(num_classes: int, n_neurons: int, n_hidden_layers: int):
    dir_path = os.getcwd()
    
    metrics_file = f"{dir_path}/training_metrics.csv"
    if os.path.exists(metrics_file):
        metrics_df = pd.read_csv(metrics_file)
    else:
        metrics_df = pd.DataFrame(columns=["Model Name", "Training Time", "Train Accuracy", "Test Accuracy"])

    file_data_train = os.path.join(dir_path, "mnist_train.csv")
    file_data_test = os.path.join(dir_path, "mnist_test.csv")
    
    data_train = pd.read_csv(file_data_train).to_numpy()
    data_test = pd.read_csv(file_data_test).to_numpy()

    x_train, y_train = data_train[:, :-1], data_train[:, -1]
    x_test, y_test = data_test[:, :-1], data_test[:, -1]

    y_train_ohe = tf.keras.utils.to_categorical(
        y_train, num_classes=num_classes)
    y_test_ohe = tf.keras.utils.to_categorical(y_test, num_classes=num_classes)

    model = tf.keras.Sequential(
        [
            tf.keras.layers.Input(shape=[x_train.shape[1]]),
        ]
    )

    for _ in range(n_hidden_layers):
        model.add(tf.keras.layers.Dense(n_neurons, activation="relu"))

    model.add(tf.keras.layers.Dense(num_classes, activation="softmax"))

    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    
    extentions = ['h5', 'keras']
    model_path = os.path.join(
        dir_path, "models", f"model_{n_hidden_layers}layers_{n_neurons}neurons"
    )
    if not os.path.join(dir_path, "models"):
        os.makedirs(os.path.join(dir_path, "models"))
    

    es = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
    ck = tf.keras.callbacks.ModelCheckpoint(
        f'{model_path}.{extentions[1]}', monitor="val_accuracy", save_best_only=True
    )

    start = time()
    model.fit(
        x_train,
        y_train_ohe,
        batch_size=4,
        epochs=100,
        validation_data=(x_test, y_test_ohe),
        verbose=2,
        callbacks=[ck, es],
    )
    end = time() 
    
    print(f"Tempo de Treinamento: {end-start}")

    # salvar modelo
    model = tf.keras.models.load_model(f'{model_path}.{extentions[1]}')

    # avaliar modelo com os dados de treinamento
    print("Resultado Treinamento")
    model.evaluate(x_train, y_train_ohe, verbose=2)

    # avaliar modelo com os dados de teste
    print("Resultado Teste")
    # model.evaluate(x_test, y_test_ohe, verbose=2)
    model.save(f'{model_path}.{extentions[0]}')
    
    train_loss, train_acc = model.evaluate(x_train, y_train_ohe, verbose=2)
    test_loss, test_acc = model.evaluate(x_test, y_test_ohe, verbose=2)
    
    metrics_df.loc[len(metrics_df)] = { # type: ignore
        "Model Name": f"model_{n_hidden_layers}layers_{n_neurons}neurons",
        "Training Time": end - start,
        "Train Accuracy": train_acc,
        "Test Accuracy": test_acc
    } 
    metrics_df.to_csv(metrics_file, index=False)

In [61]:
n_hidden_layers = 3
n_neurons = 20
num_classes = 10
gerar_rede(num_classes, n_neurons, n_hidden_layers)

Epoch 1/100
15000/15000 - 23s - 2ms/step - accuracy: 0.9002 - loss: 0.3303 - val_accuracy: 0.9408 - val_loss: 0.1974
Epoch 2/100
15000/15000 - 21s - 1ms/step - accuracy: 0.9418 - loss: 0.1962 - val_accuracy: 0.9492 - val_loss: 0.1709
Epoch 3/100
15000/15000 - 22s - 1ms/step - accuracy: 0.9498 - loss: 0.1649 - val_accuracy: 0.9503 - val_loss: 0.1666
Epoch 4/100
15000/15000 - 22s - 1ms/step - accuracy: 0.9559 - loss: 0.1480 - val_accuracy: 0.9517 - val_loss: 0.1625
Epoch 5/100
15000/15000 - 23s - 2ms/step - accuracy: 0.9587 - loss: 0.1350 - val_accuracy: 0.9595 - val_loss: 0.1400
Epoch 6/100
15000/15000 - 22s - 1ms/step - accuracy: 0.9610 - loss: 0.1274 - val_accuracy: 0.9558 - val_loss: 0.1598
Epoch 7/100
15000/15000 - 22s - 1ms/step - accuracy: 0.9638 - loss: 0.1206 - val_accuracy: 0.9520 - val_loss: 0.1667
Epoch 8/100
15000/15000 - 22s - 1ms/step - accuracy: 0.9651 - loss: 0.1146 - val_accuracy: 0.9585 - val_loss: 0.1449
Epoch 9/100
15000/15000 - 22s - 1ms/step - accuracy: 0.9666 - lo

Resultado Teste
1875/1875 - 3s - 2ms/step - accuracy: 0.9743 - loss: 0.0816
313/313 - 1s - 3ms/step - accuracy: 0.9608 - loss: 0.1438
